In [1]:
import numpy as np
import pandas as pd
import sklearn
import keras

Using TensorFlow backend.


In [146]:
movies = pd.read_csv('archive/movies_metadata.csv')

In [5]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [147]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [148]:
movies = movies.filter(['original_title', 'id', 'imdb_id','vote_average', 'vote_count'])

In [149]:
movies["reviews"] = [[] for _ in range(len(movies))]
movies.head()

,original_title,id,imdb_id,vote_average,vote_count,reviews
0,Toy Story,862,tt0114709,7.7,5415.0,[]
1,Jumanji,8844,tt0113497,6.9,2413.0,[]
2,Grumpier Old Men,15602,tt0113228,6.5,92.0,[]
3,Waiting to Exhale,31357,tt0114885,6.1,34.0,[]
4,Father of the Bride Part II,11862,tt0113041,5.7,173.0,[]


In [150]:
import json
import os
import re
import requests

CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'
REVIEW_PATTERN = 'https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={key}'
KEY = '6f2ceb0e0b459afe90f6e854fcd410eb'
            
def _get_json(url):
    r = requests.get(url)
    return r.json()


def get_tmdb_reviews():
    """ request reviews with 'id' from TMDB and append them to the reviews column """
    k=0
    for index, row in movies.iterrows():
        id = row['id']
        r = _get_json(REVIEW_PATTERN.format(key=KEY,movie_id=id))
        
        if not r:
            print("request cannot be located")
            continue
        
        n = r.get('total_results')
        
        reviews = []
        for i in range(n):
            k += 1
            result = r['results'][i]
            review = result.get('content')
            reviews.append(review)
            #rating = result.get('author_details').get('rating')
            
        movies.at[index, 'reviews'] = reviews
        
        # SET LIMIT
        if index > 20:
            print(k)
            break

    return None


get_tmdb_reviews()

13


In [151]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

import string

def tokenize(list):
    t = []
    
    for s in list:
        tokens = s.split()

        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]
        
        tokens = [word for word in tokens if word.isalpha()]
        
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]
        
        tokens = [word for word in tokens if len(word) > 1]
        
        t.append(tokens)
    
    t = [word.lower() for sl in t for word in sl]
        
    ps = PorterStemmer()
    for w in t:
        print(w)
    
    # returns a flat list of strings
    return t

def clean_data():
    for index, row in movies.iterrows():
        review = row['reviews']
        movies.at[index, 'reviews'] = tokenize(review)
        
        # SET LIMIT
        if index > 20:
            print(k)
            break
            
    return None



clean_data()
movies.head(20)

[nltk_data] Downloading package stopwords to C:\Users\Gina
[nltk_data]     Wu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


this
movie
came
three
now
im
twenty
seven
goddamn
thing
still
holds
final
very
strong
appeal
personal
favourite
decided
revisit
many
years
still
holds
well
great
movie
kids
adults
wonderful
teachable
moments
just
groundbreaking
animated
movie
around
stunning
feature
film
entrance
pixar
emtoy
storyem
true
delight
first
second
last
the
cgi
animation
excellent
all
toys
look
brilliant
world
love
feel
the
music
good
randy
newman
nice
job
youve
got
friend
me
cracker
tom
hanks
leads
strong
cast
hanks
plays
woody
fantastic
effect
he
best
part
film
tim
allen
great
buzz
lightyear
don
rickles
mr
potato
head
wallace
shawn
rex
john
ratzenberger
hamm
also
bring
fun
everything
else
terrific
humour
pacing
plot
enjoy
there
probably
flaws
parts
namely
buzzs
supposed
unawareness
nothing
impacts
viewing
experience
go
watch
throw
dice
take
turn
jumanji
made
critics
gurn
jumanji
directed
joe
johnston
based
chris
van
allsburgs
short
story
name
it
stars
robin
williams
bonnie
hunt
kirsten
dunst
bradley
pierce


,original_title,id,imdb_id,vote_average,vote_count,reviews
0,Toy Story,862,tt0114709,7.7,5415.0,"[this, movie, came, three, now, im, twenty, se..."
1,Jumanji,8844,tt0113497,6.9,2413.0,"[throw, dice, take, turn, jumanji, made, criti..."
2,Grumpier Old Men,15602,tt0113228,6.5,92.0,[]
3,Waiting to Exhale,31357,tt0114885,6.1,34.0,[]
4,Father of the Bride Part II,11862,tt0113041,5.7,173.0,[]
5,Heat,949,tt0113277,7.7,1886.0,"[you, dont, live, live, among, remains, dead, ..."
6,Sabrina,11860,tt0114319,6.2,141.0,[]
7,Tom and Huck,45325,tt0112302,5.4,45.0,"[good, enough, it, doesnt, come, close, disney..."
8,Sudden Death,9091,tt0114576,5.5,174.0,[]
9,GoldenEye,710,tt0113189,6.6,1194.0,"[really, solid, entry, series, brosnan, person..."


In [ ]:
def encode_text():
    return padded

In [138]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

def custom_model(kength, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model